# Using `AdherR` through `rpy2` from a `Python3` Jupyter Notebook

We use here `rpy2` to compute and plot CMA7 from a `Python3` `Jupyter Notebook`...

In [1]:
# import ryp2 (please note that you might need to install it as per https://rpy2.github.io/doc.html)
import rpy2

In [2]:
# check that all is ok
rpy2.__path__

['/Users/ddediu/Library/Python/3.10/lib/python/site-packages/rpy2']

In [3]:
# import R's "AdhereR" package
from rpy2.robjects.packages import importr
adherer = importr('AdhereR')

# access the internal R session
import rpy2.robjects as robjects

In [4]:
# access the med.events dataset
med_events = robjects.r['med.events']

In [5]:
# check its type
type(med_events)

rpy2.robjects.vectors.DataFrame

In [6]:
# HTML printing of data frames
import rpy2.ipython.html
rpy2.ipython.html.init_printing()

# print it
med_events

,,PATIENT_ID,DATE,PERDAY,CATEGORY,DURATION
0,286,1,04/26/2033,4,medA,50
1,287,1,07/04/2033,4,medB,30
2,288,1,08/03/2033,4,medB,30
3,289,1,08/17/2033,4,medB,30
4,291,1,10/13/2033,4,medB,30
5,290,1,10/16/2033,4,medB,30
6,292,1,11/23/2033,4,medB,30
7,293,1,01/19/2034,4,medB,30
...,...,...,...,...,...,...
1078,949,100,05/26/2034,6,medB,30


In [7]:
# make some AdhereR functions available to Python
CMA7 = robjects.r['CMA7']
getCMA = robjects.r['getCMA']

In [8]:
# create a CMA7 object
cma7 = CMA7(med_events,
            ID_colname="PATIENT_ID",
            event_date_colname="DATE",
            event_duration_colname="DURATION",
            event_daily_dose_colname="PERDAY",
            medication_class_colname="CATEGORY",
            followup_window_start=230,
            followup_window_duration=705,
            observation_window_start=41,
            observation_window_duration=100,
            date_format="%m/%d/%Y")

In [9]:
# print it for checking
cma7

0,data,...
1,ID.colname,"[1] ""PATIENT_ID"""
2,event.date.colname,"[1] ""DATE"""
3,event.duration.colname,"[1] ""DURATION"""
4,event.daily.dose.colname,"[1] ""PERDAY"""
5,medication.class.colname,"[1] ""CATEGORY"""
6,medication.groups,<rpy2.rin...
7,flatten.medication.groups,[1] FALSE
...,...,...
24,CMA,...
25,event.info,...


In [10]:
# print the estimated CMAs
getCMA(cma7)

,,PATIENT_ID,CMA
0,1,1,1.0
1,2,2,0.35
2,3,3,0.9299999999999999
3,4,4,nan
4,5,5,0.30000000000000004
5,6,6,0.06000000000000005
6,7,7,0.24
7,8,8,0.39
...,...,...,...
98,99,99,0.7


In [12]:
# plot it
# this is some clunky code involving do.call and named lists because
# plot has lots of arguments with . that cannot be autmatically handeled by rpy2
# the idea is to use a TaggedList that associated values and argument names

import rpy2.rlike.container as rlc # for TaggedList
rcall = robjects.r['do.call'] # do.call()
grdevices = importr('grDevices') # R graphics device

# the actual plotting
grdevices.jpeg(file="./cma7plot.jpg", width=512, height=512)
rcall("plot",
      rlc.TaggedList([cma7,
                      robjects.IntVector([1,2,3]), 
                      False,
                      True], 
                     tags=('cma', 
                           'patients.to.plot', 
                           'show.legend',
                           'align.all.patients')))
grdevices.dev_off()

1


Include the CMA7 plot using the Markdown syntax (there are several alternatives: https://stackoverflow.com/questions/32370281/how-to-embed-image-or-picture-in-jupyter-notebook-either-from-a-local-machine-o): 

![The plot](./cma7plot.jpg)